In [ ]:
import requests
import json
import os
from instabase.notebook.conf import ibconfig

# URI
INSTABASE_URI = 'https://staging.instabase.com'
RUN_CLUSTER_API = INSTABASE_URI + '/apps/cluster/read_files'
GET_FLOW_STATUS_API = INSTABASE_URI + '/api/v1/jobs/status'

# Set your folder paths
# ---------------------

INPUT_DIR = u'/sudeep/instabase-tutorials/fs/Instabase Drive/files/doc_A/input'

# Request header
MY_API_TOKEN = 'zr3NkK0axjXUrPEOAXB8k7TuNd79OV'                                     # Fill this out: Put your API token here.
HEADERS = {
  'Authorization': 'Bearer {}'.format(MY_API_TOKEN),
}


# Process Files
# -------------
RUN_FLOW_API = INSTABASE_URI + '/api/v1/flow/run_flow_async'

def process_files(input_dir_path):
  data = json.dumps({
    'input_dir': input_dir_path,
    'ibflow_path': 'sudeep/instabase-tutorials/fs/Instabase Drive/helper_flows/process_files_flow.ibflow',
  })
  r = requests.post(RUN_FLOW_API, headers=HEADERS, data=data)
  return r.content


import StringIO
import time

def get_status(job_id):
    r = requests.get(
        GET_FLOW_STATUS_API + '?job_id=' + job_id, headers=HEADERS, cookies=ibconfig.cookies)
    print(r.content)
    return json.loads(r.content)

def get_flow_output_folder(flow_resp):
    
    job_id = flow_resp['data']['job_id']
    
    print('Showing flow results for job: ' + job_id)
    done = False
    resp = {}
    
    while not done:
        resp = get_status(job_id)
        print(resp)
        if resp['status'] != 'OK':
            print(resp['msg'])
            return

        if resp['state'] != 'DONE':
            cur_status = json.loads(resp.get('cur_status', '{}'))
            step_name = cur_status.get('stepName')
            msg = cur_status.get('curMsg')
            print('Running: ' + step_name + ', ' + msg + '...')
            time.sleep(4)
            continue
        
        done = True
        break
    
    output_folder = resp['results'][0]['output_folder']
    return output_folder


resp_json = json.loads(process_files(INPUT_DIR))

PATH_TO_OUTPUT_OF_PROCESS_FILES = get_flow_output_folder(resp_json)

print(PATH_TO_OUTPUT_OF_PROCESS_FILES)

In [ ]:
def run_clustering():
    data = json.dumps({
        "concurrent_file_read": 25,
        "folders": "folder_1<NEWLINE>folder_2",
        "repo_name": "your_repo_name",
        "repo_owner": "<instabase_id>",
        "root_dir": "path/to/output/folder"
    })
    